# opencv-基礎教學筆記
[參考](https://medium.com/jimmy-wang/opencv-%E5%9F%BA%E7%A4%8E%E6%95%99%E5%AD%B8%E7%AD%86%E8%A8%98-with-python-d780f571a57a)

## 一、圖片與影像讀取
### 圖片讀取很簡單，但端看是否要呈現出來(show)，注意若使用 colab或遠端環境，會是遠端的資源開啟圖片，本地／連線端是看不到的。

In [1]:
# 於python環境執行指令前先導入package
import cv2 as cv

In [8]:
# 讀取圖片
img = cv.imread('Resouces/cat.jpg')
cv.imshow('Cat', img)
cv.waitKey(0)

-1

In [28]:
# 讀取影片
capture = cv.VideoCapture('Resources/dog.mp4')
while True:
    isTrue, frame = capture.read()
    if isTrue: 
        cv.imshow('Video', frame)
 
    # 讀取過程中若按下 q 則離開
    if cv.waitKey(27) & 0xFF==ord('q'):
        break 
 
    # 持續讀取影片，直到讀取完畢
    else:
        break
capture.release()
cv.destroyAllWindows()

## 二、用OpecCV繪圖
### 繪圖並不困難，困難的是參數要怎麼放，代表什麼意思。

In [2]:
# 這邊導入numpy套件
import cv2 as cv
import numpy as np

In [33]:
blank = np.zeros((500,500,3), dtype='uint8')
# 1. 畫指定位置(直接給位置參數)
blank[200:300, 300:400] = 255,255,100
cv.imshow('Cyan', blank)
#2. 畫方框(畫布, 起點, 終點, 顏色, 粗度)
# 如果 thickness=-1則為填滿
cv.rectangle(blank, (0,0), (250,250), (0,255,0), thickness = 3)
cv.imshow('Green', blank)
# 3. 畫圓形(畫布, 圓心位置, 半徑radius, 顏色, 粗度)
# 如果 thickness=-1則為填滿
cv.circle(blank, (blank.shape[1]//2, blank.shape[0]//2), 40, (0,0,255), thickness = 3)
cv.imshow('Red', blank)
# 4. 畫線條(畫布, 起點, 終點, 顏色, 粗度)
cv.line(blank ,(0,0), (blank.shape[1]//2, blank.shape[0]//2), (255,255,255), thickness = 3)
cv.imshow('Red', blank)
# 5. 加入文字(畫布, 文字內容, 起點, 字體, 大小, 顏色, 粗度)
cv.putText(blank, '中文呢? This is Jimmy.', (150,225), cv.FONT_HERSHEY_TRIPLEX, 1.0, (0,255,0), 2)  #中文不行
cv.imshow('Text', blank)
# 等待圖片關閉
cv.waitKey(0)

-1

## 三、圖像預處理與形態學(Morphology)
### 圖片預處理有很多種，包含模糊化(blur)、輪廓化(edge cascade)、膨脹、侵蝕、先膨脹後侵蝕、先侵蝕後膨脹，具有不同的效果，並且有許多小技巧可以進行調整，影片在此先介紹到最基礎的內容。

In [1]:
# 這邊導入numpy套件
import cv2 as cv
import numpy as np

In [5]:
img = cv.imread('Resouces/cat.jpg')
# 糊糊化(blur)
# 注意 kernel size必須是奇數(odd)，kernel size越大越模糊。
# 模糊化也有許多不同的方法可以進行，讓 kernel依據需求進行運算。
blur = cv.GaussianBlur(img, (7,7), cv.BORDER_DEFAULT)
cv.imshow('Blur Cats', blur)
# 輪廓化(edge cascade)
# 小技巧，可以先將圖片模糊化，再進行輪廓化，可以抓到比較少雜訊。
canny = cv.Canny(blur, 125, 175)
cv.imshow('Canny Cats', canny)
# 注意膨脹、侵蝕用的照片是已經輪廓化處理過。雜訊會較少。
# 膨脹 dilating
dilated = cv.dilate(canny, (7,7), iterations=3)
cv.imshow('Dilated', dilated)
# 侵蝕 eroding
eroded = cv.erode(canny, (3,3), iterations=1)
cv.imshow('Eroded', eroded)
cv.waitKey(0)

-1

### 移動圖片位置 (Translation)

In [14]:
img = cv.imread('Resouces/cat.jpg')
# 移動圖片位置 (Translation)
def translate(img, x, y):
    transMat = np.float32([[1,0,x],[0,1,y]])
    dimensions = (img.shape[1], img.shape[0])
    return cv.warpAffine(img, transMat, dimensions)
# -x → 往左移動 ; x → 往右移動
# -y → 往上移動 ; y → 往下移動
translated = translate(img, -100, 100)
cv.imshow('Source', img)
cv.imshow('Translated', translated)
cv.waitKey(0)

-1

### 旋轉圖片(Rotation)

In [ ]:
img = cv.imread('Resouces/cat.jpg')
# 旋轉圖片
def rotate(img, angle, rotPoint=None):
    (height,width) = img.shape[:2]
    if rotPoint is None:
        rotPoint = (width//2,height//2)
 
    rotMat = cv.getRotationMatrix2D(rotPoint, angle, 1.0)
    dimensions = (width,height)
    return cv.warpAffine(img, rotMat, dimensions)

cv.imshow('Source', img)
# 輸入角度作為參數
rotated = rotate(img, -45)
cv.imshow('Rotated', rotated)
rotated_rotated = rotate(img, -90)
cv.imshow('Rotated Rotated', rotated_rotated)
cv.waitKey(0)

## 四、圖片結構(輪廓與層級) Contours & Hierarchies
### 算是最難理解的部分，但基本上就是根據Canny輪廓化的結果(或採其他方式)，進行運算，找出輪廓裡外的關係，計算總共有幾層結構。

In [1]:
# 這邊導入numpy套件
import cv2 as cv
import numpy as np

In [5]:
img = cv.imread('Resouces/cat.jpg')
blank = np.zeros(img.shape, dtype='uint8')
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
blur = cv.GaussianBlur(gray, (5,5), cv.BORDER_DEFAULT)
canny = cv.Canny(blur, 125, 175)

contours, hierarchies = cv.findContours(canny, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
# 計算總共有幾個輪廓 contours
print(f'{len(contours)} contour(s) found!')
# 畫出當前所有的 contours
cv.drawContours(img, contours, -1, (255,0,0), 1)
cv.imshow('Contours Drawn on img', img)
# 標示 contours
cv.drawContours(blank, contours, -1, (0,255,0), 1)
cv.imshow('Contours Drawn on blank', blank)
cv.waitKey(0)
#cv.destroyAllwindows()

2793 contour(s) found!


-1

# (OpenCV)前景／背景分離技術[轉錄]
[參考](https://medium.com/linux-on-raspberry-pi4/opencv-%E5%89%8D%E6%99%AF-%E8%83%8C%E6%99%AF%E5%88%86%E9%9B%A2%E6%8A%80%E8%A1%93-%E8%BD%89%E9%8C%84-d2745e41d159)

## cv2.absdiff

### 它的功能就是將兩張相片相減後取絕對值。例如，我們將影片中兩張連續兩個frame進行cv2.absdiff，單憑肉眼似乎感覺不出前後兩張frame的差異，但使用cv2.absdiff之後便會得到如上的結果
### cv2.absdiff能夠有效的去除背景取得移動中的物體，但卻有很多的限制讓它不是那麼好用：

#### 1.必須使用兩張圖片。
#### 2.除非能夠隨時update用於diff計算的背景圖片。
#### 3.相當容易受環境變動（如光線、週遭物件、物體與背景顏色）的干擾。
#### 4.若移動中的物體與背景顏色相同則無法偵測。
#### 5.若物體靜止或移動得很緩慢，則無法偵測。

### 除了cv2.absdiff，其實OpenCV還另外提供了四種針對影片的背景分割器可用以分離影像中的前景與背景，它們分別是K-Nearest（KNN）、Mixture of Gaussians的MOG及MOG2、Geometric Multigid（GMG）等。它們的用法非常簡單，先使用cv2.createBackgroundSubtractor建立物件（可在此階段輸入參數），接著便可傳入影像使用apply命令去學習並取得去除背景後的結果。

## BackgroundSubtractorMOG

## BackgroundSubtractorMOG2

## BackgroundSubtractorGMG

## BackgroundSubtractorKNN

#### cv2.diffabs的作法是前後frame之間的差異，因此邊緣部份會比較明顯。
#### 其它MOG、MOG2、KNN及GMG四種都是使用預設參數。
#### MOG2在陰影處理上比起MOG好很多，但是在無陰影狀況下取出的物件形態似乎又沒有MOG來得好。GMG由於需要先學習，因此一開始不會顯示。